In [2]:
import polars as pl
import numpy as np
from datetime import datetime, timedelta

In [ ]:
np.random.seed(42)
def generate_coffee_data():
    n_records = 2000  # 2000 transactions over 6 months
    #Menue:
    menu_items = ['Espresso', 'Cappuccino', 'Latte', 'Americano', 'Mocha', 'Cold Brew']
    prices = [2.50, 4.00, 4.50, 3.00, 5.00, 3.50]
    # Getting dates over 6 months
    start_date = datetime(2023, 6, 1)
    dates = [start_date + timedelta(days=np.random.randint(0, 180))
            for _ in range(n_records)]